# Analysis of Ski Resort Data

## Summary

This notebook explores the data taken from www.skiresorts.info using a selenium based scraper, see https://github.com/awkirby/ski-scraping.git.

The data was partially cleaned post-scraping, Pandas and other tools (plotly, pyplot etc.) are used to complete the cleaning and visualise the data set.

The statsmodel module will then be used to fit a linear regression based model to the data in order to calculate the cost of a ski pass at each resort. 

A cost estimator could be a useful tool either for the resorts themselves or for customers to identify whether a resort is good value for money or not.

In [26]:
import pandas as pd
import numpy as np

In [27]:
# Path to csv file
path = "results/ski_resort_data_clean.csv"

# Lets start by looking at what we have
df_ski_data = pd.read_csv(path)

# Get some summary statistics
df_ski_data.describe()

,Unnamed: 0,Unnamed: 0.1,Access Order,ID,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Cost in Euros
count,3397.000000,3397.000000,3397.000000,3397.000000,3049.000000,3397.000000,3397.000000,3397.000000,3397.000000,3378.000000,3378.000000,3378.000000,3397.000000,3397.000000,3397.000000
mean,2045.925817,2045.925817,2046.925817,7359.718281,2.546146,367.386223,850.466294,1217.852517,16.178246,6.124719,6.957608,3.123653,5.595525,810.317592,31.119974
std,1316.264607,1316.264607,1316.264607,9811.071014,0.618926,371.116761,551.851679,774.428130,34.955345,14.429009,16.088254,8.214040,8.261449,4614.145507,21.298469
min,0.000000,0.000000,1.000000,1.000000,1.700000,5.000000,3.000000,25.000000,0.100000,0.000000,0.000000,0.000000,1.000000,2.000000,2.000000
25%,917.000000,917.000000,918.000000,1777.000000,2.000000,102.000000,456.000000,647.000000,1.500000,0.800000,0.500000,0.000000,2.000000,23.000000,17.000000
50%,1908.000000,1908.000000,1909.000000,3506.000000,2.400000,225.000000,764.000000,1025.000000,5.000000,2.000000,2.000000,0.500000,3.000000,44.000000,27.000000
75%,3107.000000,3107.000000,3108.000000,5929.000000,2.900000,504.000000,1195.000000,1740.000000,15.000000,5.475000,6.200000,2.600000,6.000000,175.000000,39.000000
max,5873.000000,5873.000000,5874.000000,34625.000000,4.900000,2509.000000,3290.000000,3950.000000,600.000000,312.000000,238.500000,126.000000,170.000000,85000.000000,350.000000


In [28]:
# Get some info
df_ski_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               3397 non-null   int64  
 1   Unnamed: 0.1             3397 non-null   int64  
 2   Access Order             3397 non-null   int64  
 3   ID                       3397 non-null   int64  
 4   Name                     3397 non-null   object 
 5   Continent                3397 non-null   object 
 6   Country                  3396 non-null   object 
 7   Web Link                 3397 non-null   object 
 8   Star Rating              3049 non-null   float64
 9   Elevation Change (m)     3397 non-null   int64  
 10  Base Elevation (m)       3397 non-null   int64  
 11  Max Elevation (m)        3397 non-null   int64  
 12  Total Piste Length (km)  3397 non-null   float64
 13  Blue Piste Length (km)   3378 non-null   float64
 14  Red Piste Length (km)   

We observe the following:

* There are several redundant columns probably due to the index being stored each time the data was saved and reloaded
* The following columns contain null values: Country, Star Rating, Blue/Red/Black Psite Lengths, and Photo
* Pandas lists stings as dtype "object" 

Our aim is to use resort information to estimate the price of a ski pass, therefore columns with information on the photos or links to the source data (i.e. "Web Link") are also redundant. 


## Data Cleaning

In [29]:
# Drop redundant columns
redundant = ["Unnamed: 0", "Unnamed: 0.1", "Web Link", "Photo URL", "Photo", "Page Link"]

df_ski_data.drop(columns=redundant, inplace=True)

# Review
df_ski_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Access Order             3397 non-null   int64  
 1   ID                       3397 non-null   int64  
 2   Name                     3397 non-null   object 
 3   Continent                3397 non-null   object 
 4   Country                  3396 non-null   object 
 5   Star Rating              3049 non-null   float64
 6   Elevation Change (m)     3397 non-null   int64  
 7   Base Elevation (m)       3397 non-null   int64  
 8   Max Elevation (m)        3397 non-null   int64  
 9   Total Piste Length (km)  3397 non-null   float64
 10  Blue Piste Length (km)   3378 non-null   float64
 11  Red Piste Length (km)    3378 non-null   float64
 12  Black Piste Length (km)  3378 non-null   float64
 13  Ski Lifts                3397 non-null   int64  
 14  Ski Pass Cost           

In [30]:
# The access order values are unique, use these as the index instead
df_ski_data.set_index('Access Order', inplace=True)

In [31]:
# Check data
df_ski_data.head()

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
1,119,Obertauern,Europe,Austria,4.5,683,1630,2313,100.0,61.0,35.0,4.0,26,49.0,€,49.0
2,107,Lermoos – Grubigstein,Europe,Austria,4.0,1096,1004,2100,27.3,12.8,12.6,1.9,8,49.5,€,49.5
3,72,Spieljoch – Fügen,Europe,Austria,3.7,1404,650,2054,17.1,3.6,10.8,2.7,7,59.0,€,59.0
4,40,Steinplatte/Winklmoosalm – Waidring/Reit im Winkl,Europe,Austria,4.1,1120,740,1860,42.0,18.0,22.0,2.0,14,50.0,€,50.0
5,158,Ski Juwel Alpbachtal Wildschönau,Europe,Austria,4.2,1195,830,2025,90.9,25.1,53.1,12.7,46,51.0,€,51.0


In [32]:
# From the .describe() of the data the maximum cost of 350 euros is well outside the 3rd Quartile price of 39 euros
# Have a look at these more expensive resorts
df_ski_data[df_ski_data["Cost in Euros"] > 100.0][["Name", "Country", "Ski Pass Cost", "Currency", "Cost in Euros"]]

,Name,Country,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,
67,Vail,USA,219.00,US$,184.0
71,Telluride,USA,169.00,US$,142.0
73,Beaver Creek,USA,218.00,US$,183.0
78,Mammoth Mountain,USA,209.00,US$,175.0
82,Snowmass,USA,199.00,US$,167.0
88,Breckenridge,USA,199.00,US$,167.0
94,Keystone,USA,176.00,US$,148.0
102,Killington,USA,165.00,US$,139.0
109,Winter Park Resort,USA,164.00,US$,138.0


Most of the more expensive resorts are in the USA! Certain areas in the US are known for their high prices so this is ok.

The two other expensive resorts are in Norway and the UK. 

The UK slope is a dry slope, so the price could be reasonable since these are expensive to run and tend to attract much smaller footfalls. However, having looked at the website, [telfordandwrekinleisure](https://www.telfordandwrekinleisure.co.uk/site/scripts/home_info.php?homepageID=20), it seems the wrong value has been used. It is the cost of learning to ski in a day, access is actually £14 for 2 hours. Given the challenge of converting this, it would be best to drop it.

The Norwegian one seems very extreme even for a country known for its high-cost of living! Looking at the resort website, https://raumaskisenter.com/priser/, it is clear that there is an error. The price should be 350 in Norwegian Kroner (NOK), equivalent to euro 35. A much more reasonable price!


In [33]:
# Correct outlier cost values
df_ski_data.drop(labels=4183, inplace=True)

df_ski_data.loc[2447,'Currency'] = "NOK"
df_ski_data.loc[2447,'Cost in Euros'] = 35.0

# Confirm
df_ski_data.loc[2447]

ID                          33350
Name                        Rauma
Continent                  Europe
Country                    Norway
Star Rating                   2.2
Elevation Change (m)          302
Base Elevation (m)            318
Max Elevation (m)             620
Total Piste Length (km)       4.1
Blue Piste Length (km)        1.4
Red Piste Length (km)         2.0
Black Piste Length (km)       0.7
Ski Lifts                       1
Ski Pass Cost               350.0
Currency                      NOK
Cost in Euros                35.0
Name: 2447, dtype: object

In [34]:
# Check the most common countries
df_ski_data['Country'].value_counts()

USA            404
Germany        392
Austria        329
Japan          278
Switzerland    272
              ... 
Armenia          1
Kazakhstan       1
Greenland        1
Cyprus           1
Lesotho          1
Name: Country, Length: 65, dtype: int64

In [35]:
# Countries with few entries may skew the results
df_ski_data['Country'].value_counts()[df_ski_data['Country'].value_counts() < 10]

Iceland                         9
Siberia                         9
China                           9
Netherlands                     7
Australia                       7
Hungary                         7
Bulgaria                        6
Latvia                          6
Georgia                         5
Belgium                         5
South Korea                     5
Serbia                          5
Lebanon                         4
Turkey                          4
Southern Russia                 3
Andorra                         3
Estonia                         3
Kyrgyzstan                      3
Denmark                         3
Northwest Russia                3
Ural Federal District           2
Croatia                         2
Lithuania                       2
Montenegro                      2
Azerbaijan                      2
Far Eastern Federal District    2
Israel                          1
India                           1
Iran                            1
Volga Federal 

Reviewing the countries with few resorts it looks like Russian pistes have ended up in a Russian Continent with the country replaced with regions, districts etc.

## Null Values

### Country

In [36]:
# The only NaN value in the Country column is also a Russian resort
df_ski_data[df_ski_data['Country'].isnull()]

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
2933,33810,Hvalin/Khvalyn,Europe,NaN,2.0,181,116,297,2.8,1.8,1.0,0.0,3,2500.0,RUB,28.0


In [37]:
# Replace NaN with Russia, otherwise the rest of the conversions won't work
df_ski_data.loc[2933,'Country'] = "Russia"
df_ski_data.loc[2933]

ID                                  33810
Name                       Hvalin/Khvalyn
Continent                          Europe
Country                            Russia
Star Rating                           2.0
Elevation Change (m)                  181
Base Elevation (m)                    116
Max Elevation (m)                     297
Total Piste Length (km)               2.8
Blue Piste Length (km)                1.8
Red Piste Length (km)                 1.0
Black Piste Length (km)               0.0
Ski Lifts                               3
Ski Pass Cost                      2500.0
Currency                              RUB
Cost in Euros                        28.0
Name: 2933, dtype: object

In [38]:
df_ski_data[df_ski_data["Continent"] == "Russia"]

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
284,1571,Rosa Khutor,Russia,Southern Russia,3.6,1380,940,2320,102.0,62.0,24.0,16.0,26,3200.0,RUB,36.0
434,1569,Gazprom Mountain Resort,Russia,Southern Russia,3.2,1307,949,2256,52.8,17.3,26.3,9.2,23,3200.0,RUB,36.0
458,1568,Krasnaya Polyana Resort,Russia,Southern Russia,3.2,1212,960,2172,30.0,7.5,20.0,2.5,13,3800.0,RUB,43.0
559,5071,Zavjalikha,Russia,Ural Federal District,NaN,426,414,840,22.0,NaN,NaN,NaN,4,2600.0,RUB,30.0
684,3001,Big Wood,Russia,Northwest Russia,3.0,667,380,1047,26.7,13.5,8.5,4.7,8,1600.0,RUB,18.0
1338,1812,Baikalsk – Sobolinaya,Russia,Siberia,2.6,479,525,1004,15.0,6.6,4.9,3.5,7,2100.0,RUB,24.0
1465,1638,Adzhigardak,Russia,Ural Federal District,2.5,408,275,683,21.7,12.2,5.8,3.7,8,2500.0,RUB,28.0
1574,1863,Blagodat – Belokuricha,Russia,Siberia,2.5,523,278,801,6.1,2.3,3.2,0.6,6,1200.0,RUB,14.0
1679,33650,Manzherok,Russia,Siberia,2.4,636,388,1024,1.5,1.5,0.0,0.0,3,700.0,RUB,8.0


In [39]:
# Where the continent is Russia, set the Country entries to Russia
df_ski_data["Country"][df_ski_data["Continent"] == "Russia"] = "Russia"

<ipython-input-39-fac19350e9b4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ski_data["Country"][df_ski_data["Continent"] == "Russia"] = "Russia"


In [40]:
# Where the continent is equal to "Russia", replace with "Europe"
df_ski_data["Continent"] = df_ski_data["Continent"].str.replace("Russia", "Europe")

In [41]:
# Check that the replace worked
df_ski_data["Continent"].value_counts()

Europe                   2424
North America             598
Asia                      317
Australia and Oceania      31
South America              25
Africa                      1
Name: Continent, dtype: int64

### Star Rating

Just over 10% of the Star Rating values are missing. Given the expectation that this will be a key variable in the linear regression, something must be done about this. We don't want to drop the values, since this would reduce the dataset size substantially. Using zero is not appropriate as this would impact the accuracy of the linear regression.

The safest option is to set the values equal to the current mean of **2.546146**. This could make some resorts appear better or worse than they are, but at least is shouldn't impact the overall importance of the feature.


In [42]:
# Before doing anything let's take a look
df_ski_data[df_ski_data["Star Rating"].isnull()]

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
559,5071,Zavjalikha,Europe,Russia,NaN,426,414,840,22.0,NaN,NaN,NaN,4,2600.0,RUB,30.0
665,2400,Mzaar Kfardebian,Asia,Lebanon,NaN,615,1850,2465,80.0,46.0,30.0,4.0,19,71000.0,LBP,39.0
1109,2006,Bromley Mountain,North America,USA,NaN,407,594,1001,45.0,15.0,18.0,12.0,9,91.0,US$,76.0
1202,3789,Onikoube,Asia,Japan,NaN,715,340,1055,7.0,NaN,NaN,NaN,6,4000.0,¥,31.0
1237,2489,Gerlosstein,Europe,Austria,NaN,907,929,1836,10.0,1.0,8.0,1.0,5,57.5,€,57.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,3812,Osmelakowa Dolina – Spalona,Europe,Poland,NaN,50,750,800,0.4,0.4,0.0,0.0,1,55.0,PLN,12.0
4606,5896,Långberget,Europe,Sweden,NaN,40,590,630,0.5,0.5,0.0,0.0,1,125.0,Skr,12.0
4609,29363,Monkova dolina – Ždiar,Europe,Slovakia,NaN,100,850,950,0.3,0.3,0.0,0.0,1,16.0,€,16.0


In [43]:
import plotly.graph_objects as go

In [44]:
# An initial look suggests a spread of resorts, which is promising with regards to the impact
# Lets visualise using go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_ski_data["Country"][df_ski_data["Star Rating"].isnull()], 
    y=df_ski_data["Cost in Euros"][df_ski_data["Star Rating"].isnull()], 
    name="Country vs Cost in Euros for Missing Star Rating Values", mode="markers"))

fig.show()

In [45]:
# Plot suggests that there are null values in a wide range of countries
# But some countries are particularly 'bad'
# Establish the percentage of missing ratings for each country
round((df_ski_data["Country"][df_ski_data["Star Rating"].isnull()].value_counts() 
       / df_ski_data["Country"].value_counts())
      *100,2)

Andorra                     NaN
Argentina                   NaN
Armenia                     NaN
Australia                 14.29
Austria                    0.91
Azerbaijan                  NaN
Belgium                   40.00
Bosnia and Herzegovina      NaN
Brazil                      NaN
Bulgaria                    NaN
Canada                    17.62
Chile                      7.14
China                     44.44
Croatia                     NaN
Cyprus                      NaN
Czech Republic            20.85
Denmark                     NaN
Estonia                   33.33
Finland                    6.67
France                     0.90
Georgia                     NaN
Germany                    0.51
Greece                      NaN
Greenland                   NaN
Hungary                     NaN
Iceland                     NaN
India                       NaN
Iran                        NaN
Israel                      NaN
Italy                      2.75
Japan                     28.42
Kazakhst

In [46]:
# Latvia and Lebanon are the only countries with less than 50% of the resorts rated
# These should be dropped to avoid skewing the results
df_ski_data[(df_ski_data["Country"] == "Latvia") | (df_ski_data["Country"] == "Lebanon")]

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
665,2400,Mzaar Kfardebian,Asia,Lebanon,NaN,615,1850,2465,80.0,46.0,30.0,4.0,19,71000.0,LBP,39.0
812,5053,Zaarour,Asia,Lebanon,2.9,355,1645,2000,16.0,0.3,9.2,6.5,4,70000.0,LBP,38.0
2060,4673,The Cedars,Asia,Lebanon,NaN,755,2095,2850,9.0,4.0,2.0,3.0,7,50000.0,LBP,27.0
2315,2399,Faqra,Asia,Lebanon,NaN,240,1735,1975,7.0,2.0,4.0,1.0,4,60400.0,LBP,33.0
2466,4102,Riekstu Kalns,Europe,Latvia,2.2,36,51,87,4.9,2.5,1.8,0.6,15,29.0,€,29.0
3922,32390,Kaķīškalns,Europe,Latvia,1.9,70,25,95,0.4,0.1,0.3,0.0,2,20.0,€,20.0
4266,1814,Baiļi,Europe,Latvia,1.9,21,46,67,0.8,0.6,0.2,0.0,8,15.0,€,15.0
4316,5057,Zagarkalns,Europe,Latvia,NaN,30,70,100,1.4,1.4,0.0,0.0,12,23.0,€,23.0
4321,3408,Milzkalns,Europe,Latvia,NaN,35,75,110,1.6,1.6,0.0,0.0,8,23.0,€,23.0


In [47]:
# Get the index values for Latvian and Lebanese ski resorts
to_drop = df_ski_data[(df_ski_data["Country"] == "Latvia") | (df_ski_data["Country"] == "Lebanon")].index

# Drop these 
df_ski_data.drop(labels=to_drop, inplace=True)

In [48]:
# Check China and Belgium for any funny business
df_ski_data[(df_ski_data["Country"] == "Belgium") | (df_ski_data["Country"] == "China")]

,ID,Name,Continent,Country,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Currency,Cost in Euros
Access Order,,,,,,,,,,,,,,,,
510,11009,Genting Resort Secret Garden,Asia,China,3.1,398,1702,2100,16.0,3.3,7.4,5.3,5,700.0,Ұ,90.0
809,1486,Duolemeidi Mountain Resort – Chongli,Asia,China,2.9,323,1640,1963,9.0,3.0,5.0,1.0,3,500.0,Ұ,65.0
1343,3217,Lianhuashan Resort,Asia,China,2.6,132,40,172,2.1,1.5,0.5,0.1,2,420.0,Ұ,54.0
1739,2756,Huaibei,Asia,China,2.4,154,146,300,4.5,1.4,3.1,0.0,3,178.0,Ұ,23.0
2136,3051,Ice Mountain (indoor ski area),Europe,Belgium,2.3,40,50,90,0.3,0.3,0.0,0.0,3,42.0,€,42.0
2546,3641,Nanshan,Asia,China,NaN,125,90,215,5.0,3.0,1.0,1.0,13,470.0,Ұ,61.0
2765,11132,Yuyang,Asia,China,NaN,220,75,295,4.0,2.0,1.0,1.0,6,448.0,Ұ,58.0
2852,2878,Jundushan,Asia,China,NaN,247,105,352,4.2,2.0,1.0,1.2,7,420.0,Ұ,54.0
2983,4813,Val de Wanne,Europe,Belgium,2.0,100,380,480,1.4,0.6,0.8,0.0,2,14.0,€,14.0


In [49]:
# For the remaining cases, and relying on the expectation that the Country feature is important,
# the average for each respective country shall be used to fill in the nan values.

# Loop through all the countries that have null values (use .unique() to avoid duplicates)
for country in df_ski_data["Country"][df_ski_data["Star Rating"].isnull()].unique():
    # Get the mean for that country
    mean_rating = df_ski_data["Star Rating"][df_ski_data["Country"] == country].mean()
    
    # Get the index of the resorts with null values in that country, this is quite a big construct so break it down
    bool_null_rating = df_ski_data["Star Rating"][df_ski_data["Country"] == country].isnull()
    null_index = df_ski_data[df_ski_data["Country"] == country][bool_null_rating].index
    
    # Replace the star rating with the mean value
    df_ski_data.loc[null_index, "Star Rating"] = mean_rating
    
# Check for null values
df_ski_data["Country"][df_ski_data["Star Rating"].isnull()].value_counts()

Series([], Name: Country, dtype: int64)

### Piste Length

There are 19 cases where the breakdown of the piste lenghts is missing. Given the small number, these could be dropped. 

However, its good to come up with solutions instead of simply dropping the values. Particularly since 13 of the resorts are in Japan, this could have a larger than necessary impact.

The suggested it approach is to set the value of red piste lengths equal to the total piste length and the other colours to 0. On average red piste's are the most common as observed when the describe method was used. 

In [50]:
# Loop through the indexes of all the resorts that are missing the piste length breakdowns
for index in df_ski_data[df_ski_data["Red Piste Length (km)"].isnull()].index:
    # Set the blue and black piste lengths to 0
    df_ski_data.loc[index, "Blue Piste Length (km)"] = 0.0
    df_ski_data.loc[index, "Black Piste Length (km)"] = 0.0
    
    # Set the Red Piste Length equal to the total piste length
    df_ski_data.loc[index, "Red Piste Length (km)"] = df_ski_data.loc[index, "Total Piste Length (km)"]

    
# Confirm there are no nulls
df_ski_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3386 entries, 1 to 5874
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       3386 non-null   int64  
 1   Name                     3386 non-null   object 
 2   Continent                3386 non-null   object 
 3   Country                  3386 non-null   object 
 4   Star Rating              3386 non-null   float64
 5   Elevation Change (m)     3386 non-null   int64  
 6   Base Elevation (m)       3386 non-null   int64  
 7   Max Elevation (m)        3386 non-null   int64  
 8   Total Piste Length (km)  3386 non-null   float64
 9   Blue Piste Length (km)   3386 non-null   float64
 10  Red Piste Length (km)    3386 non-null   float64
 11  Black Piste Length (km)  3386 non-null   float64
 12  Ski Lifts                3386 non-null   int64  
 13  Ski Pass Cost            3386 non-null   float64
 14  Currency                

## Visualisation

Visualise the current values to identify any further issues.

In [51]:
import plotly.express as px

In [52]:
# Create a violin plot of the Country against the ski pass cost.
# No obvious outliers are observed
fig = px.violin(df_ski_data, y="Cost in Euros", x="Country", box=True, points="all", title="Country vs Ski Pass Cost")
fig.update_layout(xaxis_type="category", xaxis={'categoryorder':'mean ascending'})

In [53]:
# Look at Star Rating against the Cost in Euros
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_ski_data["Star Rating"], 
    y=df_ski_data["Cost in Euros"], 
    name="Star Rating vs Cost in Euros", mode="markers"))

fig.update_layout(title="Star Rating vs Cost in Euros", xaxis_title="Star Rating",
    yaxis_title="Cost in Euros")

fig.show()

The trend is mixed, but there is clearly a corellation. Albeit one that does not apply equally.

In [54]:
# Look at Total Piste Length against the Cost in Euros
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_ski_data["Total Piste Length (km)"], 
    y=df_ski_data["Cost in Euros"], 
    name="Piste Length vs Cost in Euros", mode="markers"))

fig.update_layout(title="Piste Length vs Cost in Euros", xaxis_title="Total Piste Length (km)",
    yaxis_title="Cost in Euros")

fig.show()

There are certainly outliers, but the cost does appear to be strongly correlated with length.

In [55]:
# Save the clean data for reference
df_ski_data.to_csv("results/ski_resort_data_LR_ready.csv")

In [24]:
# Review the correlations in the data
df_ski_data.corr()

,ID,Star Rating,Elevation Change (m),Base Elevation (m),Max Elevation (m),Total Piste Length (km),Blue Piste Length (km),Red Piste Length (km),Black Piste Length (km),Ski Lifts,Ski Pass Cost,Cost in Euros
ID,1.000000,-0.425803,-0.334806,-0.219465,-0.316863,-0.236689,-0.202385,-0.228816,-0.207901,-0.243774,-0.058608,-0.332134
Star Rating,-0.425803,1.000000,0.814925,0.344191,0.642132,0.713185,0.612401,0.692845,0.607678,0.651131,0.057452,0.695134
Elevation Change (m),-0.334806,0.814925,1.000000,0.384220,0.753090,0.656842,0.577516,0.651462,0.513907,0.594383,0.046828,0.506370
Base Elevation (m),-0.219465,0.344191,0.384220,1.000000,0.896769,0.290974,0.207138,0.283410,0.325243,0.166448,0.060484,0.304683
Max Elevation (m),-0.316863,0.642132,0.753090,0.896769,1.000000,0.522170,0.424547,0.514327,0.478117,0.403494,0.065546,0.459819
Total Piste Length (km),-0.236689,0.713185,0.656842,0.290974,0.522170,1.000000,0.904514,0.952962,0.811088,0.863349,0.004619,0.543310
Blue Piste Length (km),-0.202385,0.612401,0.577516,0.207138,0.424547,0.904514,1.000000,0.778924,0.576838,0.881557,0.005240,0.364633
Red Piste Length (km),-0.228816,0.692845,0.651462,0.283410,0.514327,0.952962,0.778924,1.000000,0.738883,0.806014,-0.003499,0.519848
Black Piste Length (km),-0.207901,0.607678,0.513907,0.325243,0.478117,0.811088,0.576838,0.738883,1.000000,0.557644,0.020036,0.660101
Ski Lifts,-0.243774,0.651131,0.594383,0.166448,0.403494,0.863349,0.881557,0.806014,0.557644,1.000000,0.036827,0.384789


In [57]:
px.imshow(df_ski_data.corr(), color_continuous_scale='Agsunset', title="Correlation heatmap of Ski Resort Data")

Ski lifts highly correlated with piste lengths as expected


## Train/Test Data

The aim is to use the features of the ski resorts to estimate the ski pass costs. The first step shall be to split the data into training and test sets. 
The sklearn module can be used to facilitate this.

In [111]:
from sklearn.model_selection import train_test_split

In [74]:
# Stasmodels package doesn't accept spaces or brackets in the column names, these will need to be replaced
df_ski_data.columns = df_ski_data.columns.str.replace(" ", "_", regex=False)
df_ski_data.columns = df_ski_data.columns.str.replace("(", "", regex=False)
df_ski_data.columns = df_ski_data.columns.str.replace(")", "", regex=False)  
df_ski_data.columns

Index(['ID', 'Name', 'Continent', 'Country', 'Star_Rating',
       'Elevation_Change_m', 'Base_Elevation_m', 'Max_Elevation_m',
       'Total_Piste_Length_km', 'Blue_Piste_Length_km', 'Red_Piste_Length_km',
       'Black_Piste_Length_km', 'Ski_Lifts', 'Ski_Pass_Cost', 'Currency',
       'Cost_in_Euros'],
      dtype='object')

In [114]:
# Because we will be using statsmodel for the linear regression we actually want to keep the labels with the feature data
X_train, X_test, y_train, y_test = train_test_split(df_ski_data, df_ski_data['Cost_in_Euros'], test_size=0.3)

In [115]:
# Lets do a few checks to see how well split the data is
round(X_train['Country'].value_counts()/len(X_train['Country'])*100, 2)

USA                       11.94
Germany                   11.43
Austria                    9.79
Japan                      8.19
Switzerland                8.14
Italy                      6.62
France                     6.54
Czech Republic             6.03
Canada                     5.57
Norway                     4.56
Sweden                     4.47
Poland                     3.00
Slovakia                   2.70
Finland                    1.69
Spain                      0.97
Slovenia                   0.97
Russia                     0.76
Romania                    0.72
New Zealand                0.63
Chile                      0.51
United Kingdom             0.42
Ukraine                    0.38
Greece                     0.38
Iceland                    0.34
Bosnia and Herzegovina     0.30
China                      0.30
Argentina                  0.25
Hungary                    0.21
Australia                  0.21
Bulgaria                   0.17
Belgium                    0.17
South Ko

In [116]:
round(X_test['Country'].value_counts()/len(X_test['Country'])*100, 2)

USA                       11.91
Germany                   11.91
Austria                    9.55
Japan                      8.27
Switzerland                7.78
Czech Republic             6.69
France                     6.69
Italy                      6.00
Canada                     6.00
Norway                     4.13
Poland                     3.74
Sweden                     3.64
Slovakia                   2.85
Finland                    1.97
United Kingdom             1.08
Slovenia                   0.98
New Zealand                0.89
Spain                      0.69
Bosnia and Herzegovina     0.59
Netherlands                0.39
Russia                     0.39
Argentina                  0.39
Romania                    0.30
Ukraine                    0.30
Turkey                     0.30
Hungary                    0.20
Serbia                     0.20
China                      0.20
Chile                      0.20
Georgia                    0.20
Australia                  0.20
Bulgaria

Looking at the percentages the split appears even.

## Linear Regression

In [1]:
import statsmodels.formula.api as smf

Strong correlations observed to piste lengths, star rating and elevation change as anticipated.

In [118]:
# Select columns to fit to model, categorical variables will be identified automatically
# Exclude Max Elevation as discussed earlier.
features = ["Continent", "Country", "Star_Rating", "Elevation_Change_m", "Base_Elevation_m", "Total_Piste_Length_km",
             "Blue_Piste_Length_km", "Red_Piste_Length_km", "Black_Piste_Length_km", "Ski_Lifts"]

In [119]:
resort_model = smf.ols("Cost_in_Euros ~ " + " + ".join(features), X_train).fit()
resort_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Cost_in_Euros   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                     128.9
Date:                Tue, 06 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:37:38   Log-Likelihood:                -8856.2
No. Observations:                2370   AIC:                         1.783e+04
Df Residuals:                    2311   BIC:                         1.817e+04
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                            -13.4070      2.591     -5.174      0.000     -18.489      -8.325
Continent[T.Asia]                     -6.5477      2.724     -2.404      0.016     -11.889      -1.207
Continent[T.Australia and Oceania]    15.9226      2.109      7.548      0.000      11.786      20.059
Continent[T.Europe]                  -14.0327      5.354     -2.621      0.009     -24.531      -3.534
Continent[T.North America]             5.6521      1.494      3.784      0.000       2.723       8.582
Continent[T.South America]            -7.7733      2.947     -2.637      0.008     -13.553      -1.994
Country[T.Argentina]                 -15.0411      4.214     -3.569      0.000     -23.305      -6.778
Country[T.Australia]                  10.3053      3.217      3.204      0.001       3.997      16.614
Country[T.Austria]                    12.1124      6.056      2.000      0.046       0.237      23.987
Country[T.Azerbaijan]                -25.1728      9.669     -2.604      0.009     -44.133      -6.213
Country[T.Belgium]                    16.2667      7.947      2.047      0.041       0.683      31.851
Country[T.Bosnia and Herzegovina]      0.1898      7.164      0.026      0.979     -13.859      14.239
Country[T.Brazil]                     -3.5613      7.747     -0.460      0.646     -18.754      11.631
Country[T.Bulgaria]                   -0.9015      7.907     -0.114      0.909     -16.406      14.603
Country[T.Canada]                     -4.6974      0.986     -4.765      0.000      -6.631      -2.764
Country[T.Chile]                      10.8291      3.676      2.946      0.003       3.621      18.038
Country[T.China]                      35.4085      4.261      8.310      0.000      27.052      43.765
Country[T.Croatia]                     3.6392      9.453      0.385      0.700     -14.898      22.176
Country[T.Czech Republic]              8.7078      6.103      1.427      0.154      -3.261      20.677
Country[T.Denmark]                    16.1883      9.486      1.707      0.088      -2.414      34.790
Country[T.Estonia]                    21.9475      9.480      2.315      0.021       3.358      40.537
Country[T.Finland]                    26.0872      6.273      4.159      0.000      13.786      38.388
Country[T.France]                      4.2148      6.052      0.696      0.486      -7.654      16.084
Country[T.Georgia]                   -31.6633      5.897     -5.370      0.000     -43.227     -20.100
Country[T.Germany]                    11.0195      6.077      1.813      0.070      -0.898      22.937
Country[T.Greece]                     -5.2000      6.911     -0.752      0.452     -18.752       8.352
Country[T.Hungary]                     8.9717      7.594      1.181      0.238      -5.921      

The initial attempt lead to a reasonable value of Rsquared being achieved of 0.758. The categorical data in Continent and Country has been identified correctly. Surprisingly the number of ski lifts does not appear to be significant. This may be a consequence of this significance being captured in the piste lengths.

Next step is to use the ANOVA1 test to confirm that the country variable is significant (ANOVA2 is unnecessary since its only one variable). This is not a question for the Continent variable since every entry had a low p-value.

In [120]:
resort_model_mean = smf.ols("Cost_in_Euros ~ 1", X_train).fit()
resort_model_country = smf.ols("Cost_in_Euros ~ Country", X_train).fit()
# Look at the residuals - They clearly demonstrate a big impact
print("Residuals when fitting the mean: ", resort_model_mean.ssr, "\nResiduals when fitting Country: ", resort_model_country.ssr)

Residuals when fitting the mean:  1034567.1398829116 
Residuals when fitting Country:  665392.3157531667


In [80]:
import statsmodels.api as sm

In [121]:
sm.stats.anova_lm(resort_model_country, typ=1)

,df,sum_sq,mean_sq,F,PR(>F)
Country,51.0,369174.824130,7238.722042,25.21724,3.899619e-183
Residual,2318.0,665392.315753,287.054493,NaN,NaN


In [122]:
# Having been convinced country is doing what it should be doing, now go back to developing the linear regression

# Look at the residuals over the resorts
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=X_train.index, 
    y=resort_model.resid, 
    name="Model Residuals", mode="markers"))

fig.update_layout(title="Model Residuals", xaxis_title="Resorts",
    yaxis_title="Residuals")

fig.show()

In [128]:
# Look at the residuals over the Star Ratings
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=X_train["Star_Rating"], 
    y=resort_model.resid, 
    name="Model Residuals", mode="markers"))

fig.update_layout(title="Model Residuals", xaxis_title="Star Ratings",
    yaxis_title="Residuals")

fig.show()

In [123]:
# Now try without the ski lifts
features.remove("Ski_Lifts")

resort_model = smf.ols("Cost_in_Euros ~ " + " + ".join(features), X_train).fit()

In [124]:
# As expected the changes are minor
resort_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Cost_in_Euros   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                     131.1
Date:                Tue, 06 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:39:29   Log-Likelihood:                -8856.6
No. Observations:                2370   AIC:                         1.783e+04
Df Residuals:                    2312   BIC:                         1.816e+04
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                            -13.3634      2.591     -5.158      0.000     -18.444      -8.283
Continent[T.Asia]                     -6.6282      2.722     -2.435      0.015     -11.966      -1.290
Continent[T.Australia and Oceania]    15.8475      2.108      7.518      0.000      11.714      19.981
Continent[T.Europe]                  -13.6627      5.338     -2.559      0.011     -24.131      -3.194
Continent[T.North America]             5.4816      1.482      3.698      0.000       2.575       8.388
Continent[T.South America]            -7.7894      2.947     -2.643      0.008     -13.569      -2.010
Country[T.Argentina]                 -14.8685      4.210     -3.532      0.000     -23.123      -6.614
Country[T.Australia]                  10.3085      3.217      3.205      0.001       4.000      16.617
Country[T.Austria]                    11.5857      6.028      1.922      0.055      -0.236      23.407
Country[T.Azerbaijan]                -25.4121      9.665     -2.629      0.009     -44.365      -6.460
Country[T.Belgium]                    15.7505      7.927      1.987      0.047       0.206      31.295
Country[T.Bosnia and Herzegovina]     -0.2950      7.145     -0.041      0.967     -14.305      13.715
Country[T.Brazil]                     -3.7465      7.744     -0.484      0.629     -18.933      11.440
Country[T.Bulgaria]                   -1.3181      7.893     -0.167      0.867     -16.797      14.161
Country[T.Canada]                     -4.8356      0.974     -4.963      0.000      -6.746      -2.925
Country[T.Chile]                      10.8256      3.676      2.945      0.003       3.617      18.034
Country[T.China]                      35.4003      4.261      8.308      0.000      27.044      43.756
Country[T.Croatia]                     3.0920      9.434      0.328      0.743     -15.408      21.592
Country[T.Czech Republic]              8.2107      6.079      1.351      0.177      -3.711      20.132
Country[T.Denmark]                    15.5974      9.464      1.648      0.099      -2.961      34.156
Country[T.Estonia]                    21.3304      9.456      2.256      0.024       2.788      39.873
Country[T.Finland]                    25.6112      6.251      4.097      0.000      13.352      37.870
Country[T.France]                      3.7299      6.029      0.619      0.536      -8.093      15.553
Country[T.Georgia]                   -31.5485      5.895     -5.351      0.000     -43.109     -19.988
Country[T.Germany]                    10.4795      6.049      1.733      0.083      -1.382      22.341
Country[T.Greece]                     -5.5965      6.897     -0.811      0.417     -19.122       7.929
Country[T.Hungary]                     8.5308      7.579      1.126      0.260      -6.331      

In [132]:
# Plot the fitted values against the actual values

#X_train["fitted_costs"] = resort_model.fittedvalues

fig = go.Figure()
# Note that we still plot against the original Year variable
fig.add_trace(go.Scatter(
    x=X_train["Cost_in_Euros"], y=resort_model.fittedvalues, name="Actual vs Fitted Costs", mode="markers"))
#fig.add_trace(go.Scatter(
   # x=X_train.index, y=X_train["fitted_costs"], name="Predicted Costs", mode="markers"))
fig.update_layout(title="Predicted Costs vs Actual Costs", xaxis_title="Actual Costs in Euros",
    yaxis_title="Predicted Costs in Euros")
fig.show()

In [141]:
print(abs(resort_model.resid).mean())
np.sqrt((resort_model.resid ** 2).mean())

6.571818138932157


10.155696089570275

In [129]:
# There is an issue with predicting the test data because not all the countries were included in the training data.
resort_model.predict(X_test.drop(columns="Ski_Lifts"))

PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error converting data to categorical: observation with value 'Portugal' does not match any of the expected levels (expected: ['Andorra', 'Argentina', ..., 'United Arab Emirates', 'United Kingdom'])
    Cost_in_Euros ~ Continent + Country + Star_Rating + Elevation_Change_m + Base_Elevation_m + Total_Piste_Length_km + Blue_Piste_Length_km + Red_Piste_Length_km + Black_Piste_Length_km
                                ^^^^^^^